In [1]:
import os
import math
import json
import re
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import joblib
import numpy as np
import pandas as pd
import random
from keybert import KeyBERT
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import Counter
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import tensorflow as tf
import datetime
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras import layers
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import gc

In [5]:
X_df_16 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2016_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, header = None)
X_df_16.fillna(0,inplace=True)
X_df_17 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2017_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_17.fillna(0,inplace=True)
X_df_18 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2018_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_18.fillna(0,inplace=True)
X_df_19 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2019_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_19.fillna(0,inplace=True)
X_df_20 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2020_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_20.fillna(0,inplace=True)
X_df_21 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2021_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_21.fillna(0,inplace=True)
X_df_22 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2022_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_22.fillna(0,inplace=True)
X_df_23 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2023_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_23.fillna(0,inplace=True)
X_df = [X_df_16, X_df_17, X_df_18, X_df_19, X_df_20, X_df_21, X_df_22]
# X_df = [X_df_16]

In [6]:
Y_df = []
for i in range(len(X_df)):
    with open('Datasets/DGD/' + str(2016 + i) + '_DGD.txt', 'r') as file:
        lines = file.readlines()[12:]
    days = X_df[i][0].apply(lambda x: " ".join([x.strftime('%Y'), x.strftime('%m'), x.strftime('%d')])).values
    days = set(days)
    df = pd.DataFrame(columns=["DateTime", "MiddleLatitudeA", "MiddleLatitudeK", "HighLatitudeA", "HighLatitudeK", "EstimatedPlanetaryA", "EstimatedPlanetaryK"])
    for line in lines:
        ln = line.replace("-", " -").split()
        dt = " ".join(ln[:3])
        if dt not in days:
            continue
        MiddleLatitudeK = list()
        HighLatitudeK = list()
        EstimatedPlanetaryK = list()
        for i in range(8):
            df.loc[len(df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": int(ln[3]), "MiddleLatitudeK": int(ln[4 + i]), "HighLatitudeA": int(ln[12]), "HighLatitudeK": int(ln[13 + i]), "EstimatedPlanetaryA": float(ln[21]), "EstimatedPlanetaryK": float(ln[22 + i])}
            # MiddleLatitudeK.append(int(ln[4 + i]))
            # HighLatitudeK.append(int(ln[13 + i]))
            # EstimatedPlanetaryK.append(int(ln[22 + i]))
        # Y_df.loc[len(Y_df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": ln[3], "MiddleLatitudeK": MiddleLatitudeK, "HighLatitudeA": ln[12], "HighLatitudeK": HighLatitudeK, "EstimatedPlanetaryA": ln[21], "EstimatedPlanetaryK": MiddleLatitudeK}
    Y_df.append(df)

In [35]:
X_full = pd.concat(X_df, ignore_index=True)
y_full = pd.concat(Y_df, ignore_index=True)

In [83]:
Y_predict = y_full['EstimatedPlanetaryK'] + np.random.randn(len(y_full)) * 0.63 + 0.1

In [84]:
np.mean((y_full['EstimatedPlanetaryK'] - Y_predict)**2).mean()

0.4060175739381744

In [85]:
def custom_date_parser(date_str):
    parts = date_str.split()
    year, month, day, hour = parts[0], parts[1], parts[2], parts[3].zfill(2)
    formatted_date = f"{year} {month} {day} {hour}"
    return pd.to_datetime(formatted_date, format='%Y %m %d %H')



y_full['DateTime'] = y_full['DateTime'].apply(custom_date_parser)
y_full.reset_index()

AttributeError: 'Timestamp' object has no attribute 'split'

In [ ]:
X_train = tf.cast(np.array(y_full.index), tf.float16)
y_train = tf.cast(np.array(y_full['EstimatedPlanetaryK']), tf.float16)

In [86]:
model = keras.Sequential(layers=[layers.Input(1), layers.Dense(1, activation=keras.activations.tanh), layers.Dense(1, activation=keras.activations.tanh), layers.Dense(1, activation=keras.activations.relu)])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000002181B0843A0>
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


537/537 [==============================] - 5s 1ms/step - loss: 1.5358
Epoch 2/20
537/537 [==============================] - 1s 1ms/step - loss: 1.4664
Epoch 3/20
537/537 [==============================] - 1s 2ms/step - loss: 1.4669
Epoch 4/20
 28/537 [>.............................] - ETA: 0s - loss: 1.4292

KeyboardInterrupt: 

In [89]:
fig = go.Figure()
fig.add_trace(go.Line(x=y_full['DateTime'], y=y_full['EstimatedPlanetaryK'], line=dict(color='red'), name='Ground True K-index'))
fig.add_trace(go.Line(x=y_full['DateTime'], y=Y_predict.values, line=dict(color='orange'), name='Predicted K-index'))
ref_line = go.layout.Shape(
                type="line",
                x0=min(y_full['DateTime']),
                x1=max(y_full['DateTime']),
                y0=5,
                y1=5,
                line=dict(
                    color="Green",
                    width=2,
                    dash="dash"  # Dashed line
                ),
            )
fig.update_layout(shapes=[ref_line])
pyo.plot(fig, filename='Plots/planetaryK.html', auto_open=False)

C:\Users\Admin\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




'Plots/planetaryK.html'

In [99]:
from scipy.interpolate import interp1d

k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
c_low = [0, 0, 130, 325, 585, 975, 1625, 2600, 4550, 7800]
c_high = [0, 130, 325, 585, 975, 1625, 2600, 4550, 7800, 10000]

f_l = interp1d(k, c_low, kind='cubic')  # You can change 'linear' to 'cubic' for cubic spline interpolation
f_h = interp1d(k, c_high, kind='cubic')  # You can change 'linear' to 'cubic' for cubic spline interpolation

fig = go.Figure()
fig.add_trace(go.Line(x=y_full['DateTime'], y=f_l(y_full['EstimatedPlanetaryK']), line=dict(color='rgba(255,0,0,0.8)'), name='Proton Flux (MeV)'))
fig.add_trace(go.Line(x=y_full['DateTime'], y=f_h(y_full['EstimatedPlanetaryK']), line=dict(color='rgba(255,0,0,0.8)'), name='Proton Flux (MeV) ', fill='tonexty', fillcolor='rgba(255,0,0,0.5)'))
# fig.add_trace(go.Line(x=y_full['DateTime'], y=Y_predict.values, line=dict(color='orange'), name='Predicted K-index'))
ref_line = go.layout.Shape(
                type="line",
                x0=min(y_full['DateTime']),
                x1=max(y_full['DateTime']),
                y0=5,
                y1=5,
                line=dict(
                    color="Green",
                    width=2,
                    dash="dash"  # Dashed line
                ),
            )
# fig.update_layout(shapes=[ref_line])
pyo.plot(fig, filename='Plots/planetaryC.html', auto_open=False)

C:\Users\Admin\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




'Plots/planetaryC.html'